<a href="https://colab.research.google.com/github/alexandergribenchenko/Data_Science_Toolkit/blob/main/Exchange/AV_Test_Entrega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
path_github = 'https://raw.githubusercontent.com/alexandergribenchenko/Data_Science_Toolkit/main/Exchange/entrega_sample.json'

In [3]:
import pandas as pd

In [5]:
df = pd.read_json(path_github, orient='records', lines=True)
df

,id_alert,snapshot_date,flight_date,rt,region,hub,airline_code,asks
0,20220227ADZAXMVH202203,20220227,202203,ADZAXM,DOM_CO,EXBOGDOM,VH,399761
1,20220227ADZMIAAA202201,20220227,202201,ADZMIA,NAMSAM,EXBOGINT,AA,2010045
2,20220227ADZPTYCM202201,20220227,202201,ADZPTY,CAMSAM,EXBOGINT,CM,587707
3,20220227ADZPVA9R202201,20220227,202201,ADZPVA,DOM_CO,EXBOGDOM,9R,219914
4,20220227AMSBOGKL202201,20220227,202201,AMSBOG,EURALL,BOGINT,KL,67586230
...,...,...,...,...,...,...,...,...
2916,20230129RIHYULTS202311,20230129,202311,RIHYUL,CAMNAM,OTHS,TS,13230559
2917,20230129RIHYYZTS202312,20230129,202312,RIHYYZ,CAMNAM,OTHS,TS,6228406
2918,20230129RTBYYZWS202312,20230129,202312,RTBYYZ,CAMNAM,OTHS,WS,3276573
2919,20230129SJOYULTS202311,20230129,202311,SJOYUL,CAMNAM,OTHS,TS,14526969
